<a href="https://colab.research.google.com/github/mrm8488/shared_colab_notebooks/blob/master/custom_learningtopaint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ICCV2019-Learning to Paint

## [Arxiv](https://arxiv.org/abs/1903.04411) | [YouTube](https://youtu.be/YmOgKZ5oipk) | [Reddit](https://www.reddit.com/r/reinforcementlearning/comments/b5lpfl/learning_to_paint_with_modelbased_deep/)

## Abstract

We show how to teach machines to paint like human painters, who can use a
small number of strokes to create fantastic paintings. By employing a neural
renderer in model-based Deep Reinforcement Learning (DRL), our agents learn to
determine the position and color of each stroke and make long-term plans to
decompose texture-rich images into strokes. Experiments demonstrate that
excellent visual effects can be achieved using hundreds of strokes. The
training process does not require the experience of human painters or stroke
tracking data.

* Our ICCV poster
  <div>
<a target="_blank" rel="noopener noreferrer" href="https://github.com/hzwer/ICCV2019-LearningToPaint/blob/master/demo/lisa.gif"><img src="https://github.com/hzwer/ICCV2019-LearningToPaint/raw/master/demo/lisa.gif" alt="Demo" style="max-width:100%;"></a><a target="_blank" rel="noopener noreferrer" href="/hzwer/ICCV2019-LearningToPaint/blob/master/demo/sunrise.gif"><img src="https://github.com/hzwer/ICCV2019-LearningToPaint/raw/master/demo/sunrise.gif" alt="Demo" style="max-width:100%;"></a><a target="_blank" rel="noopener noreferrer" href="/hzwer/ICCV2019-LearningToPaint/blob/master/demo/sunflower.gif"><img src="https://github.com/hzwer/ICCV2019-LearningToPaint/raw/master/demo/sunflower.gif" alt="Demo" style="max-width:100%;"></a>
<a target="_blank" rel="noopener noreferrer" href="/hzwer/ICCV2019-LearningToPaint/blob/master/demo/palacemuseum.gif"><img src="https://github.com/hzwer/ICCV2019-LearningToPaint/raw/master/demo/palacemuseum.gif" alt="Demo" style="max-width:100%;"></a><a target="_blank" rel="noopener noreferrer" href="https://github.com/hzwer/ICCV2019-LearningToPaint/blob/master/demo/deepdream_night.gif"><img src="https://github.com/hzwer/ICCV2019-LearningToPaint/raw/master/demo/deepdream_night.gif" alt="Demo" style="max-width:100%;"></a><a target="_blank" rel="noopener noreferrer" href="/hzwer/ICCV2019-LearningToPaint/blob/master/demo/deepdream_bird.gif"><img src="https://github.com/hzwer/ICCV2019-LearningToPaint/raw/master/demo/deepdream_bird.gif" alt="Demo" style="max-width:100%;"></a>
</div>

>Colab customized by [mrm8488](https://twitter.com/mrm8488)

# 1. Clone the project

In [0]:
!git clone https://github.com/hzwer/LearningToPaint.git

Cloning into 'LearningToPaint'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 429 (delta 12), reused 12 (delta 4), pack-reused 404
Receiving objects: 100% (429/429), 18.65 MiB | 19.35 MiB/s, done.
Resolving deltas: 100% (228/228), done.


In [0]:
cd LearningToPaint/

/content/LearningToPaint


In [0]:
!mkdir uploaded_images

# 2. Download the renderer and the actor

In [0]:
#@title Choose a stroke pattern (renderer/actor)

option = 'Default' #@param ["Default", "Triangle", "Round", "Bezierwotrans"]
print(option)

Bezierwotrans


In [0]:
urls = {
    "Default" : {"renderer": "https://drive.google.com/uc?export=download&id=1-7dVdjCIZIxh8hHJnGTK-RA1-jL1tor4", "actor": "https://drive.google.com/uc?export=download&id=1a3vpKgjCVXHON4P7wodqhCgCMPgg1KeR" },
		"Triangle" : {"renderer": "https://drive.google.com/uc?authuser=0&id=1YefdnTuKlvowCCo1zxHTwVJ2GlBme_eE&export=download", "actor": "https://drive.google.com/uc?authuser=0&id=1k8cgh3tF7hKFk-IOZrgsUwlTVE3CbcPF&export=download" },
		"Round" : {"renderer": "https://drive.google.com/uc?authuser=0&id=1kI4yXQ7IrNTfjFs2VL7IBBL_JJwkW6rl&export=download", "actor": "https://drive.google.com/uc?authuser=0&id=1ewDErUhPeGsEcH8E5a2QAcUBECeaUTZe&export=download" },
		"Bezierwotrans" : {"renderer": "https://drive.google.com/uc?authuser=0&id=1XUdti00mPRh1-1iU66Uqg4qyMKk4OL19&export=download", "actor": "https://drive.google.com/uc?authuser=0&id=1VBtesw2rHmYu2AeJ22XvTCuzuqkY8hZh&export=download" }
}

renderer_url = urls[option]["renderer"]
print(renderer_url)
actor_url = urls[option]["actor"]
print(actor_url)

https://drive.google.com/uc?authuser=0&id=1XUdti00mPRh1-1iU66Uqg4qyMKk4OL19&export=download
https://drive.google.com/uc?authuser=0&id=1VBtesw2rHmYu2AeJ22XvTCuzuqkY8hZh&export=download


In [0]:
!wget "$renderer_url" -O renderer.pkl

In [0]:
!wget "$actor_url" -O actor.pkl

In [0]:
from google.colab import files

<a id='start'></a>

# 3. Remove existing files and upload ours one

In [0]:
!rm -rf ./output/*
![ -e video.mp4 ] && rm video.mp4

In [0]:
uploaded = files.upload()
pic_name = list(uploaded.keys())[0]
print(pic_name)
!mv ./$pic_name ./uploaded_images/$pic_name

Saving 30062009104.jpg to 30062009104.jpg
30062009104.jpg


# 4. Create the painting's strokes
PS: It can take a while

In [0]:
!python3 baseline/test.py --max_step=80 --actor=actor.pkl --renderer=renderer.pkl --img=uploaded_images/$pic_name --divide=5

# 5. Convert it to video

In [0]:
!ffmpeg -r 30 -f image2 -i output/generated%d.png -s 512x512 -c:v libx264 -pix_fmt yuv420p video.mp4 -q:v 0 -q:a 0

# 6. Watch the video generated

In [0]:
from IPython.display import display, Image
import moviepy.editor as mpy
display(mpy.ipython_display('video.mp4', height=256, max_duration=100.))

# 7. Download it!

In [0]:
files.download("./video.mp4")

### If you want to try with another picture go to step 3 or if you wanna try with another stroke pattern (round, triangle, etc) start from step 2

 Did you enjoy? Buy me a coffe :)
 
[![ko-fi](https://www.ko-fi.com/img/githubbutton_sm.svg)](https://ko-fi.com/Y8Y3VYYE)